# 1 Import Libraries

In [227]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

print("Evironment is ready for the project!")

Evironment is ready for the project!


# 2 Data Loading

In [20]:
movies = pd.read_csv ('D:\\My\\SLIIT\\Projects\\Movie Recommendation System\\ml-latest-small\\movies.csv')
ratings = pd.read_csv ('D:\\My\\SLIIT\\Projects\\Movie Recommendation System\\ml-latest-small\\ratings.csv')

In [26]:
print ("Movies Data Set:\n")
movies.head()

Movies Data Set:



,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [28]:
print("Ratings Data Set:\n")
ratings.head()

Ratings Data Set:



,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


# 3 Exploratory Data Analysis (EDA)

## (Exploring Data)

In [34]:
print ("Movies Columns: ", movies.columns)
print ("Movies Shape:", movies.shape)

print ("\nRatings Columns:", ratings.columns)
print ("Ratings Shape:", ratings.shape)

Movies Columns:  Index(['movieId', 'title', 'genres'], dtype='object')
Movies Shape: (9742, 3)

Ratings Columns: Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
Ratings Shape: (100836, 4)


In [52]:
print("Movies Summary:\n\n", movies.describe())
print ("\n\n\nRatings Summary:\n\n", ratings.describe())

Movies Summary:

              movieId
count    9742.000000
mean    42200.353623
std     52160.494854
min         1.000000
25%      3248.250000
50%      7300.000000
75%     76232.000000
max    193609.000000



Ratings Summary:

               userId        movieId         rating     timestamp
count  100836.000000  100836.000000  100836.000000  1.008360e+05
mean      326.127564   19435.295718       3.501557  1.205946e+09
std       182.618491   35530.987199       1.042529  2.162610e+08
min         1.000000       1.000000       0.500000  8.281246e+08
25%       177.000000    1199.000000       3.000000  1.019124e+09
50%       325.000000    2991.000000       3.500000  1.186087e+09
75%       477.000000    8122.000000       4.000000  1.435994e+09
max       610.000000  193609.000000       5.000000  1.537799e+09


# 4 Build Recommendation Models

## (Implement and test different recommendation models)

### 4.1 Data Cleaning

In [89]:
print("Movies missing values:\n")
movies.isnull().sum()

Movies missing values:



movieId    0
title      0
genres     0
dtype: int64

In [91]:
print("Ratings missing values:\n")
ratings.isnull().sum()

Ratings missing values:



userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [95]:
movies = movies.dropna()  ### drop missing values if it has.
ratings = ratings.dropna()

In [109]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


### 4.2 Merge Data

In [126]:
data = pd.merge(movies, ratings, on = 'movieId')    ### creating new data frame for merge data
data.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5


### 4.3 Analyze data

In [147]:
# Top Rated Movies

top_rated_movies = data.groupby('title')['rating'].mean().sort_values(ascending=False).head(10)
print("Top Rated Movies:\n")
top_rated_movies

Top Rated Movies:



title
Gena the Crocodile (1969)                    5.0
True Stories (1986)                          5.0
Cosmic Scrat-tastrophe (2015)                5.0
Love and Pigeons (1985)                      5.0
Red Sorghum (Hong gao liang) (1987)          5.0
Thin Line Between Love and Hate, A (1996)    5.0
Lesson Faust (1994)                          5.0
Eva (2011)                                   5.0
Who Killed Chea Vichea? (2010)               5.0
Siam Sunset (1999)                           5.0
Name: rating, dtype: float64

In [151]:
# Most Rated Movies

most_rated_movies = data.groupby('title')['rating'].count().sort_values(ascending = False).head(10)
print("Most Rated Movies:\n\n",most_rated_movies)

Most Rated Movies:

 title
Forrest Gump (1994)                          329
Shawshank Redemption, The (1994)             317
Pulp Fiction (1994)                          307
Silence of the Lambs, The (1991)             279
Matrix, The (1999)                           278
Star Wars: Episode IV - A New Hope (1977)    251
Jurassic Park (1993)                         238
Braveheart (1995)                            237
Terminator 2: Judgment Day (1991)            224
Schindler's List (1993)                      220
Name: rating, dtype: int64


### 4.4 Simple Popularity Based Recommender

#### (Recommend Top Rated or most Rated Movies)

In [170]:
### Calculate average number and number of ratings for each movie

ratings_average = ratings.groupby('movieId')['rating'].mean()
ratings_count = ratings.groupby('movieId')['rating'].count()

In [181]:
### Merge the result with movie dataset

#copy the data into another data frame. so that, original one is still same and another copied data frame will be modified
movies_with_ratings = movies.copy()

movies_with_ratings['avg_ratings'] = movies_with_ratings['movieId'].map(ratings_average)
movies_with_ratings['num_ratings'] = movies_with_ratings['movieId'].map(ratings_count)

In [189]:
### Filter Movies

popular_movies = movies_with_ratings[movies_with_ratings['num_ratings']>50]
popular_movies = movies_with_ratings.sort_values(by= ['num_ratings', 'avg_ratings'], ascending = False)

In [199]:
### Display top 10 popular movies

print("Top 10 popular movies")
popular_movies[['title', 'avg_ratings', 'num_ratings']].head(10)

Top 10 popular movies


,title,avg_ratings,num_ratings
314,Forrest Gump (1994),4.164134,329.0
277,"Shawshank Redemption, The (1994)",4.429022,317.0
257,Pulp Fiction (1994),4.197068,307.0
510,"Silence of the Lambs, The (1991)",4.161290,279.0
1939,"Matrix, The (1999)",4.192446,278.0
224,Star Wars: Episode IV - A New Hope (1977),4.231076,251.0
418,Jurassic Park (1993),3.750000,238.0
97,Braveheart (1995),4.031646,237.0
507,Terminator 2: Judgment Day (1991),3.970982,224.0
461,Schindler's List (1993),4.225000,220.0


### Colloborative Filtering 

### Recommend movies based on user ratings and similarities

#### 1. Prepare the Data

In [213]:
# Create User Item Matrix
user_item_matrix = ratings.pivot_table (index='userId', columns= 'movieId', values='rating')

#display first rows
user_item_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 2. Calculate similarity between movies

In [215]:
# Fill NaN values with 0
user_item_matrix = user_item_matrix.fillna(0)

In [219]:
user_item_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [229]:
# Calculate cosine similarity between movies
movie_similarity = cosine_similarity(user_item_matrix.T)

In [235]:
# Convert similarity matrix to a dataframe
movie_similarity_df = pd.DataFrame (movie_similarity, index = user_item_matrix.columns, columns = user_item_matrix.columns)

In [245]:
# Display the similarity of a movie
movie_similarity_df[1].sort_values(ascending = False).head(10)

movieId
1       1.000000
3114    0.572601
480     0.565637
780     0.564262
260     0.557388
356     0.547096
364     0.541145
1210    0.541089
648     0.538913
1265    0.534169
Name: 1, dtype: float64

#### 3. Make Recommendations 

In [253]:
# Function for recommend movies for a user
def recommend_movies(user_id, user_item_matrix, movie_similarity_df, top_n = 10):
    
    ## get the movies rated by the user
    rated_movies = user_item_matrix.loc [user_id]
    rated_movies = rated_movies[rated_movies > 0].index.tolist()
    
    ## Calculate weighted similarity scores
    movie_scores = pd.Series()
    
    for movieId in rated_movies:
        similar_movies = movie_similarity_df['movieId']
        movie_scores = movie_scores.add(similar_movies, fill_value = 0)

    ## Sort movies based on similarity score and exclude already rated movies
    movie_scores = movies_score.sort_values (Asceding = False)
    movie_scores = movie_scores[~movie_scores.index.isin(rated_movies)]   ### Exclude already rated movies

